This notebook generates a csv file that contains tweets about covid-19, plus other important informations such as the number of retweets, likes and replies, datetime ... 

In [4]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import os
from datetime import datetime
pd.set_option('display.max_colwidth', None)


## Load data 

In [2]:
## Number of pickle files
n_pickles=27  
## The path where the pickles are stored. 
path='/home/hamza/projects/filiere/data'

with open(os.path.join(path,'0.pickle'),'rb') as pkl :
    sample0=pickle.load(pkl)

## save into a dataframe 
df_all=pd.DataFrame(sample0)
for n_pickle in range(1,n_pickles):
    with open(os.path.join(path,'{}.pickle'.format(str(n_pickle))),'rb') as pkl :
        sample=pickle.load(pkl)
    df_all=pd.concat([df_all,pd.DataFrame(sample)],ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'projects/filiere/data\\0.pickle'

In [5]:
## Get an idea about the resulted dataframe
df_all.head()

lang  \
0   en   
1   en   
2   en   
3   en   
4   en   

                                                                                                                                                                                                                                                                                     text  \
0                                                                                                                   Only 2 out of 100 motorists agreed to move their cars from a wheelchair spot when asked by a wheelchair user.\nThere is an epidemic of selfishness in Irish motoring.   
1  An important distinction, especially for those who weren't born yet, or weren't old enough to remember what it was like during the height of #AIDS epidemic homophobia.\nIt wasn't just unpopular to be supportive of #LGBTQ #gayrights -- it was dangerous, politically & physically.   
2                           Measles has killed more than 6,000 people in the Democratic Republic of Congo, the World Health Organization says.\nThe WHO called on international partners and agencies to increase resources to fight what it called the "world's worst measles epidemic."   
3                                                                                                                                                                                               If we all talked to our kids this way, maybe we'd end the 'Anxiety Epidemic' in our youth   
4                             Me: Vacant housing is an epidemic.\n2016 Politicians: There's no vacant homes in Canada, Stephen. Stop writing about it, you're lying.\n2020 Politicians: Vacant home taxes! It's outrageous how many vacant homes are in Canada.\n¯\_(ツ)_/¯\n#onpoli #ToRE   

      timestamp         username  \
0  1.578505e+09  DublinCommuters   
1  1.578510e+09       VertDuFerk   
2  1.578505e+09             cnni   
3  1.578490e+09    MelMacEachern   
4  1.578490e+09   StephenPunwasi   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Get informations 

The idea here is to get all possible informations from the HTML file. Unfortunately, it does not contain the location. So
to study for instance the french population, we will assume at this point that all french tweets concern French pepole. 
Nevertheless, we provide another notebook that explores twitter api in order to get users locations. 


In [ ]:
def get_infos(x): 
    """
    Function that parses html file using BeautifulSoup in order to get these infos : number of replies, retweets, likes 
    and if the tweet is a share tweet or not .
    """
    soup = BeautifulSoup(x, 'html.parser')
    all_divs=soup.find_all('div')
    all_labels=[link.get('aria-label') for link in all_divs]
    
    replies,retweets,likes,done=None,None,None,False
    for x in all_labels :
        if x==None :
            continue 
        if done==False and (('like') in x or ('reply') in x or 'Retweet' in x ):
            try : 
                replies=int(re.findall('([0-9]+) repl',x)[0])
            except :
                replies=0
            try : 
                retweets=int(re.findall('([0-9]+) Retweet',x)[0])
            except :
                retweets=0 
            try : 
                likes=int(re.findall('([0-9]+) like',x)[0])
            except :
                likes=0 
            done=True
        if 'share' in x :
            test=True
        else : 
            test=False
    return ,replies,retweets,likes,test

In [ ]:
tqdm.pandas()
tabs_add = df_all['html'].progress_apply(lambda x:get_infos(x))  

## Convert the data obtained via the parsing into a dataframe
df_new = pd.DataFrame(list(tabs_add.values), columns =['datetime', 'replies', 'retweets','likes','is_share']) 
## Add the columns
df_all= pd.concat( [df_all,df_new] ,axis=1)

## get datetime
df_all['datetime']=df_all['timestamp'].apply(lambda x:datetime.fromtimestamp(x))

del df_all['html'],df_all['timestamp']

## view the five first example with head 
df_all.head()

In [ ]:
## save into a pickle 
path_to_save='/home/hamza/projects/filiere/saved_data/df_processed.pickle'
pickle.dump(df_all,open(path_to_save,'wb'))

## Preprocessing 

In [2]:
df_tweets=pickle.load(open(path_to_save,'rb'))

In [22]:
### Define cleaning function 

dict_sub = {
    "\n": " ",
    "\x89Û_": "",  # Special characters
    "\x89ÛÒ": "",
    "\x89ÛÓ": "",
    "\x89ÛÏWhen": "When",
    "\x89ÛÏ": "",
    "\x89Û÷": "",
    "\x89Ûª": "",
    "\x89Û\x9d": "",
    "å_": "",
    "\x89Û¢": "",
    "\x89Û¢åÊ": "",
    "åÊ": "",
    "åÈ": "",
    "Ì©": "e",
    "å¨": "",
    "&gt;": ">",  # Character entity references
    "&lt;": "<",
    "&amp;": "&",
    "http\S+|www.\S+": "",
    "@[A-Za-z0-9]+":""
}
dict_replace = {"...": " ... ", "'": " ' "}


def clean(tweet):
    #tweet=p.clean(tweet)
    for key, val in dict_sub.items():
        tweet = re.sub(r"{}".format(key), val, tweet)
    for key, val in dict_replace.items():
        tweet = tweet.replace(r"{}".format(key), val).strip()

    # Punctuations at the start or end of words
    for punctuation in "@!?()[]*%":
        tweet = tweet.replace(punctuation, " {} ".format(punctuation)).strip()
    
    tweet=tweet.replace('#','')
    if "http" not in tweet:
        tweet = tweet.replace(":", " : ").strip()
        tweet = tweet.replace(".", " . ").strip()
    return tweet

In [23]:
## Apply on French 
df_french=df_tweets[df_tweets.lang=='fr']
df_french['text']=df_french['text'].apply(lambda x: clean(x))


/home/hamza/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
df_french.to_csv('/home/hamza/projects/filiere/saved_data/df_french_clean.csv',index=False)